In [ ]:
# Food Web Simulation with the Niche Model and Lotka-Volterra Dynamics

This notebook builds a food web using the niche model (Williams & Martinez, 2000) and simulates the population dynamics using a Lotka-Volterra system.

---

## 1. Import Required Packages

```python
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
```

---

## 2. Define Functions

```python
def generate_niche_model(S, C):
    """
    Generate a food web using the niche model.

    Parameters:
        S (int): Number of species
        C (float): Desired connectance (fraction of realized links)

    Returns:
        np.ndarray: Adjacency matrix where entry (i, j) = 1 if species i eats species j.
    """
    niche_values = np.sort(np.random.uniform(0, 1, S))
    beta = (1 / (2 * C)) - 1
    r_values = np.random.beta(1, beta, S) * niche_values
    centers = np.random.uniform(r_values / 2, niche_values)

    adjacency_matrix = np.zeros((S, S))
    for i in range(S):
        for j in range(S):
            if abs(niche_values[j] - centers[i]) < r_values[i] / 2:
                adjacency_matrix[i, j] = 1
    return adjacency_matrix

def lotka_volterra(t, x, r, A):
    """
    Lotka-Volterra ODEs.

    Parameters:
        t (float): Time (required by solve_ivp)
        x (np.ndarray): Species abundances
        r (np.ndarray): Intrinsic growth rates
        A (np.ndarray): Interaction matrix

    Returns:
        np.ndarray: Time derivatives (dx/dt)
    """
    dxdt = x * (r + A @ x)
    return dxdt
```

---

## 3. Set Parameters

```python
S = 15        # Number of species
C = 0.15      # Connectance
Tmax = 50     # Maximum simulation time
```

---

## 4. Generate the Food Web

```python
adjacency = generate_niche_model(S, C)

# Build interaction matrix A:
# Positive effect on predator, negative effect on prey
A = np.zeros((S, S))
for i in range(S):
    for j in range(S):
        if adjacency[i, j]:
            A[i, j] = -0.1   # Predator harms prey
            A[j, i] =  0.1   # Prey benefits predator

# Add self-regulation (intraspecific competition)
np.fill_diagonal(A, -1.0)

# Intrinsic growth rates
r = np.random.uniform(0.5, 1.5, S)
for i in range(S):
    if not adjacency[:, i].any():
        r[i] = np.random.uniform(0.5, 1.5)  # Basal species
    else:
        r[i] = np.random.uniform(-1.5, -0.5)  # Consumers
```

---

## 5. Set Up and Solve the Lotka-Volterra Equations

```python
# Initial conditions
x0 = np.random.uniform(0.1, 1.0, S)

# Solve the system
sol = solve_ivp(
    fun=lambda t, x: lotka_volterra(t, x, r, A),
    t_span=(0, Tmax),
    y0=x0,
    dense_output=True,
    method='RK45'
)
```

---

## 6. Plot Species Dynamics

```python
t = np.linspace(0, Tmax, 500)
X = sol.sol(t)

plt.figure(figsize=(10,6))
for i in range(S):
    plt.plot(t, X[i], label=f'Species {i+1}')
plt.xlabel('Time')
plt.ylabel('Abundance')
plt.title('Lotka-Volterra Dynamics of a Food Web')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
```

---

## 7. Plot the Food Web Structure

```python
plt.figure(figsize=(6,6))
plt.imshow(adjacency, cmap='Greys', interpolation='none')
plt.title('Food Web Adjacency Matrix')
plt.xlabel('Prey')
plt.ylabel('Predator')
plt.colorbar(label='Link Presence')
plt.show()
```

---

## 8. Supplementary Analyses

### 8.1 Species Persistence

```python
final_abundances = X[:, -1]
persistence_threshold = 1e-3
persisting_species = np.sum(final_abundances > persistence_threshold)

print(f"\nNumber of persisting species: {persisting_species} out of {S}")
```

### 8.2 Final Abundance Distribution

```python
plt.figure(figsize=(8,5))
plt.hist(final_abundances, bins=10, edgecolor='black')
plt.xlabel('Final Abundance')
plt.ylabel('Number of Species')
plt.title('Histogram of Final Abundances')
plt.show()
```

### 8.3 Basic Network Metrics

```python
num_links = np.sum(adjacency)
connectance_actual = num_links / (S*S)

basal_species = np.where(np.sum(adjacency, axis=1) == 0)[0]
intermediate_species = np.where((np.sum(adjacency, axis=1) > 0) & (np.sum(adjacency, axis=0) > 0))[0]
top_species = np.where(np.sum(adjacency, axis=0) == 0)[0]

print(f"Actual connectance: {connectance_actual:.3f}")
print(f"Basal species: {len(basal_species)}")
print(f"Intermediate species: {len(intermediate_species)}")
print(f"Top species: {len(top_species)}")
```
